In [ ]:
import json
import matplotlib.pyplot as plt
import numpy as np
from datetime import datetime, timedelta, timezone
import requests
import os
from time import sleep
import pytz
from scipy.stats import pearsonr
from Functions import check_correlation, analyze_events, wrap_analyze_events, load_data, data_preparation, wrap_analyze_events_multiprocessing, show_output, start_wrap_analyze_events_multiprocessing
import pandas as pd
pd.set_option('display.max_rows', None)
from time import time
from multiprocessing import Process
from multiprocessing import Lock, Pool, Manager
from time import time

In [ ]:
start_interval=datetime(2023,5,7)
end_interval=datetime.now()

filter_position=(0,100)

data, df = load_data(start_interval=start_interval, end_interval=end_interval, filter_position=filter_position)
df

In [ ]:
# WRAP ANALYSIS EVENTS
list_buy_vol = ['buy_vol_5m', 'buy_vol_15m', 'buy_vol_30m', 'buy_vol_60m', 'buy_vol_24h']
list_buy_vol = ['buy_vol_5m', 'buy_vol_15m', 'buy_vol_30m', 'buy_vol_60m']
list_vol = ['vol_5m', 'vol_15m', 'vol_30m', 'vol_60m', 'vol_24h']
list_vol = ['vol_5m', 'vol_15m', 'vol_30m', 'vol_60m']
list_minutes = [5, 15, 30, 60, 120, 180, 360]
list_event_buy_volume = [0.7, 0.8]
list_event_volume = [5,6]

resp = wrap_analyze_events(data, list_buy_vol, list_vol, list_minutes, list_event_buy_volume,
                    list_event_volume, start_interval, end_interval)


In [ ]:
list_buy_vol = ['buy_vol_5m', 'buy_vol_15m', 'buy_vol_30m', 'buy_vol_60m', 'buy_vol_24h']
list_buy_vol = ['buy_vol_5m', 'buy_vol_15m', 'buy_vol_30m', 'buy_vol_60m']
list_vol = ['vol_5m', 'vol_15m', 'vol_30m', 'vol_60m', 'vol_24h']
list_vol = ['vol_5m', 'vol_15m', 'vol_30m', 'vol_60m']
list_minutes = [5, 15, 30, 60, 120, 180, 360]
list_event_buy_volume = [0.7, 0.8]
list_event_volume = [5,6]
n_processes = 6

shared_data = start_wrap_analyze_events_multiprocessing(data, list_buy_vol, list_vol, list_minutes, list_event_buy_volume, list_event_volume, start_interval, end_interval, n_processes)

In [ ]:
# ANALYSIS MULTIPROCESSING 2
t1 = time()

data_arguments = data_preparation(data, n_processes = 6)

# MULTIPROCESSING WRAP ANALYSIS EVENTS
list_buy_vol = ['buy_vol_5m', 'buy_vol_15m', 'buy_vol_30m', 'buy_vol_60m', 'buy_vol_24h']
list_buy_vol = ['buy_vol_5m', 'buy_vol_15m', 'buy_vol_30m', 'buy_vol_60m']
list_vol = ['vol_5m', 'vol_15m', 'vol_30m', 'vol_60m', 'vol_24h']
list_vol = ['vol_5m', 'vol_15m', 'vol_30m', 'vol_60m']
list_minutes = [5, 15, 30, 60, 120, 180, 360]
list_event_buy_volume = [0.7, 0.8]
list_event_volume = [5,6]

total_combinations = len(list_buy_vol) * len(list_vol) * len(list_minutes) * len(list_event_volume) * len(list_event_buy_volume)
print('total_combinantions', ': ', total_combinations)
path = "/home/alberto/Docker/Trading/analysis/analysis_json/"



now = datetime.now()
now_year = str(now.year)
now_month = str(now.month)
now_day = str(now.day)
now_hour = str(now.hour)
now_minute = str(now.minute)
file_path = path + now_month+now_day+ '_' + now_hour+now_minute + '.json'

manager = Manager()
# Create shared memory for JSON data
shared_data = manager.Value(str, json.dumps({}))
lock = Manager().Lock()


# Create a multiprocessing Pool
pool = Pool()

# Execute the function in parallel
pool.starmap(wrap_analyze_events_multiprocessing, [(arg, arg_i, list_buy_vol, list_vol, list_minutes,
                                    list_event_buy_volume, list_event_volume, start_interval,
                                    end_interval, file_path, lock, shared_data) for arg, arg_i in zip(data_arguments, range(1,len(data_arguments)+1))])

# Close the pool
pool.close()
pool.join()
t2 = time()
print(t2-t1, ' seconds')


In [ ]:
shared_data.value

In [ ]:
def show_output(shared_data):
    '''
    This function takes as input the shared_data from "wrap_analyze_events_multiprocessing" ans return the output available for pandas
    '''
    shared_data = json.loads(shared_data.value)
    output = {}
    for key in list(shared_data.keys()):
        if key is not 'coins' or key is not 'events':
            if shared_data[key]['events'] > 0:
                shared_data[key]['price_changes'] = np.array(shared_data[key]['price_changes'])

                isfinite = np.isfinite(shared_data[key]['price_changes'])
                shared_data[key]['price_changes'] = shared_data[key]['price_changes'][isfinite]

                mean_weighted = np.mean(shared_data[key]['price_changes'])*100
                std_weighted = np.std(shared_data[key]['price_changes'])*100

                output[key] = {'mean': mean_weighted, 'std': std_weighted, 'n_coins': len(shared_data[key]['coins']), 'n_events': shared_data[key]['events']}
            else:
                output[key] = {'mean': None, 'std': None, 'n_coins': 0, 'n_events': 0}

    return output

In [ ]:
output = show_output(shared_data)
df = pd.DataFrame(output).transpose()
lb = []
for index, row in df.iterrows():
    mean = row['mean']
    std = row['std']
    lb.append(mean - std)
    
    
df['lb'] = lb
    
df.sort_values(by=['lb'], ascending=False)

In [ ]:
df = pd.DataFrame(resp).transpose()
lb = []
for index, row in df.iterrows():
    mean = row['mean']
    std = row['std']
    lb.append(mean - std)
    
    
df['lb'] = lb
    
df.sort_values(by=['lb'], ascending=False)

In [ ]:
output = show_output(shared_data)
df = pd.DataFrame(output).transpose()
df